In [ ]:
import os, sys, random, re, collections, string

import numpy as np
import re

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import BCELoss

from tqdm.auto import tqdm

import sklearn.model_selection
import sklearn.metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import heapq

import matplotlib.pyplot as plt

import transformers
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

try:
    from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset


In [ ]:
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained('distilgpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('distilbert/distilgpt2')

# https://huggingface.co/datasets/cnn_dailymail
dataset = load_dataset('cnn_dailymail', '3.0.0')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when poss

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [35]:
#you can then get the individual splits
train = dataset['train']
test = dataset['test']
validation = dataset['validation']

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [32]:
ctr = len(train)
i = 0
j = 0
for dat in train:
    i+=1
    # print(i, len(dat['article'].split(' -- ')[1]))
    # print(i, dat['article'].count('--'))
    splitstr = re.split(r' -- ', dat['article'], 1)
    if (re.search(r'(\([\w+]+\))|([A-Z][A-Z]+)|(\([\w\s]+\))|(\([\w]+\.[\w]+\))', splitstr[0])) is not None:
        # print(i, splitstr)
        j+=1
    if i > ctr:
        break

print(j, j/ctr)

266071 0.9267117824689234


In [ ]:
class cnnDataSet(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.id = self.data['id']
        self.article = self.data['article']
        self.highlight = self.data['highlight']

    def get_article(text):
        splittext = text.split(" -- ")[1]




In [ ]:
def trainModel(model):
    pass

In [57]:
def split_article(art):
    splitstr = re.split(' -- ', art, 1)
    # print(splitstr)
    if len(splitstr) == 1 or (re.search(r'(\([\w+]+\))|([A-Z][A-Z]+)|(\([\w\s]+\))|(\([\w]+\.[\w]+\))', splitstr[0])) is None:
        return art
    else:
        return splitstr[1]


def tokenize_str(examples):
    # inputs = [doc for doc in examples["article"]]
    inputs = []
    labs = examples['highlights']
    # print(len(labs))
    # print(len(examples["article"]))
    ctr = 0
    for i, x in enumerate(examples["article"]):
        art = split_article(x)

        inputs.append(art)

    model_inputs = gpt2_tokenizer(inputs, max_length=1024, truncation=True)

    labels = gpt2_tokenizer(text_target=labs, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_train_dataset = train.map(tokenize_str, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]